In [8]:
#default_exp search_glovo

In [9]:
#export
import pandas as pd
import numpy as np
from food.tools import *
from food.psql import *
from food.paths import *
from sqlalchemy import Table,BIGINT,Column
from qdrant_client import QdrantClient
from food.qdrant import *
from qdrant_client.http.models import Filter, FieldCondition, Range

import numpy as np
import requests

project_name = 'glovo'
client = dev_client

In [10]:
#export
def prep_params(params):
    df = pd.DataFrame([params]).T.reset_index()
    params_df = df['index'].str.split('_', expand=True)
    params_df.columns = ['cond','key']
    return params_df.join(df[0].to_frame('value')).sort_values('cond')
    
def prep_consitions(params):
    params = prep_params(params)
    keys = params['key'].unique()
    conditions = []

        
    for key in keys:
        param = params[params['key'] == key]

        if param['cond'].tolist() == ['max','min']: # both min and max prepsent
            conditions.append(FieldCondition(
                    key=key, 
                    range=Range(
                        gte=param[param['cond']=='min']['value'].iloc[0],
                        lte=param[param['cond']=='max']['value'].iloc[0]
                    )
                ))

        elif param['cond'].tolist() == ['max']: # both min and max prepsent
            conditions.append(FieldCondition(
                    key=key, 
                    range=Range(
                        lte=param[param['cond']=='max']['value'].iloc[0]
                    )
                ))

        elif param['cond'].tolist() == ['min']: # both min and max prepsent
            conditions.append(FieldCondition(
                    key=key, 
                    range=Range(
                        gte=param[param['cond']=='min']['value'].iloc[0]
                    )
                ))

    return conditions

In [11]:
#export
def search_by_clip(collection_name,clip, topk,params={}):

    # query_filter = Filter(must=prep_consitions(params)) if len(params)>0 else None
    results = client.search(collection_name=collection_name,query_vector=clip,top=topk)#,query_filter=query_filter)
    return [r.score for r in results], [r.id for r in results]

In [12]:
#export
def search_by_text(text,topk=5,params={},prompt = None,prompt_factor=3,collection_name=project_name,return_clip = False):
    clip = np.array(requests.post(f'https://guru.skynet.center/text2vector/?text={text}').json())
    if prompt: 
        prompt_clip = np.array(requests.post(f'https://guru.skynet.center/text2vector/?text={prompt+text}').json())
        diff = prompt_clip - clip
        clip = clip + diff*prompt_factor
    results = search_by_clip(collection_name,clip, topk,params)
    df = get_metadata(results[1],return_clip)
    df['accuracy'] =results[0]
    df['url'] = df.apply(lambda x: f"http://glovo.away.guru/photos_resized/{x['path']}",axis=1)

    return df
id_table = Table('ids_meta',Base.metadata,Column('id', BIGINT),extend_existing=True)

In [13]:
#export
def get_metadata(ids,return_clip = False):
    session = Session()
    session.execute('CREATE TEMPORARY TABLE ids_meta(id bigint) ON COMMIT DROP')
    stmt = id_table.insert([{'id':t} for t in ids])
    session.execute(stmt)

    if return_clip:
        q = f"""select country_code,city_code,store_name,product_name,collection_section,product_description, path, clip
                FROM {project_name}.photos p
                join {project_name}.clips c on (p.id = c.id)
                INNER JOIN ids_meta             m  ON (p.id =      m.id)"""

    else:
        q = f"""select country_code,city_code,store_name,product_name,collection_section,product_description, path
                                    FROM {project_name}.photos p
                                    INNER JOIN ids_meta             m  ON (p.id =      m.id)"""

    df = pd.read_sql(q,session.connection())
    session.close()
    return df


In [14]:
project_name = 'glovo'

In [16]:
client = dev_client

In [19]:
search_by_text('poop',topk=100,params={},prompt = None,prompt_factor=3,collection_name=project_name,return_clip = False)

,country_code,city_code,store_name,product_name,collection_section,product_description,path,accuracy,url
0,IT,MIL,ODStore Duomo,"Marshmallow ""Che Giornata"" gr.10",Caramelle e chewingum,None,JLXSHBF_0004188_16074033.png,0.271631,http://glovo.away.guru/photos_resized/JLXSHBF_...
1,IT,TOR,Maestro - Gelato e Caffè Italiano,"Birra Bacialè ""birrificio Sagrin""",Le bibite di Maestro,Birra stagionale con menta di Pancalieri e sco...,TTGXCGR_0020239_1539119948.png,0.251559,http://glovo.away.guru/photos_resized/TTGXCGR_...
2,IT,MIL,AS_131,"CACCA SQUEEZE, CA. 7 X 6 CM",VARIE,"CACCA SQUEEZE, CA. 7 X 6 CM",JLXSHBF_0039894_1588221012.png,0.246853,http://glovo.away.guru/photos_resized/JLXSHBF_...
3,PE,LIM,La Lucha Sanguchería Criolla,Pan campesino (1 kg.),Panadería La Lucha,None,VJTTJQD_0057091_1156941710.png,0.241773,http://glovo.away.guru/photos_resized/VJTTJQD_...
4,EC,VDC,Tía,Chupetes Plop 480 G Fresa Acida,Chupetes Y Paletas,None,CKBSJCC_0074318_1523249810.png,0.236935,http://glovo.away.guru/photos_resized/CKBSJCC_...
...,...,...,...,...,...,...,...,...,...
95,PT,OPE,Cepsa Loja Conveniência,TOALHITAS DODOT 54 UNI,APROVEITE,None,MZTZMDV_0002992_1367962034.png,0.207288,http://glovo.away.guru/photos_resized/MZTZMDV_...
96,TR,IZR,Glovo Bakkal,Dr. Oetker Kabartma Tozu 10g x 5,Tatlı,None,QLVFZJM_0005673_390407524.png,0.207232,http://glovo.away.guru/photos_resized/QLVFZJM_...
97,IT,MIL,Market Milano,Carta igienica 1 rotolo,Essentials,None,JLXSHBF_0016352_1135973309.png,0.207204,http://glovo.away.guru/photos_resized/JLXSHBF_...
98,UA,BOR,Fora / ФОРА,Сардельки Фарро з молоком н/о 1г 100г,Варені ковбасні вироби,None,XHXZMCJ_0002064_1175279278.png,0.207172,http://glovo.away.guru/photos_resized/XHXZMCJ_...


In [7]:
client.get_collections()

['food_categories', 'glovo', 'food_images']

In [7]:
collection_name = 'glovo'

In [34]:
search_by_text('poop',return_clip=True)

,country_code,city_code,store_name,product_name,collection_section,product_description,path,clip,accuracy,url
0,EC,VDC,Tía,Chupetes Plop 480 G Fresa Acida,Chupetes Y Paletas,None,CKBSJCC_0074318_1523249810.png,"[0.018002281, 0.019293046, -0.000352613, 0.026...",0.236935,http://glovo.away.guru/photos_resized/CKBSJCC_...
1,GE,TBI,AS_005,აბადოკი Shitheads,წვეულების სხვა აქსესუარები,None,QSMFWRR_0471772_2291863639.png,"[0.027430251, 0.034310345, -0.01238888, 0.0478...",0.225134,http://glovo.away.guru/photos_resized/QSMFWRR_...
2,PL,WAW,AS_309,Worki Na Odchody 8 Rolek,Akcesoria,None,XMJCLQB_0023605_1580020769.png,"[0.02092579, 0.029475963, -0.016862512, 0.0533...",0.223929,http://glovo.away.guru/photos_resized/XMJCLQB_...
3,PT,EPO,note!,Estojo Quadrado Tie Dye Oh My Pop,"Mochilas, Estojos e Acessórios",None,NCQPRGV_0001828_2291375511.png,"[0.010928093, 0.016359286, 0.003763708, 0.0420...",0.223450,http://glovo.away.guru/photos_resized/NCQPRGV_...
4,BA,SRV,Fine Food Alipašino polje,"Kraljevska kifla 0,50gr",Peciva,None,FZKQYZG_0008681_1728603731.png,"[-0.03893969, 0.014553349, 0.00199332, 0.01722...",0.222521,http://glovo.away.guru/photos_resized/FZKQYZG_...


In [22]:
!nbdev_build_lib

Converted 00_clipmodel.ipynb.
Converted 00_custom_pandas.ipynb.
Converted 00_paths.ipynb.
Converted 00_progress_check.ipynb.
Converted 00_psql.ipynb.
Converted 00_qdrant.ipynb.
Converted 00_search_glovo.ipynb.
Converted 00_tools.ipynb.
Converted 01_multiple_foods.ipynb.
Converted 01_search.ipynb.
Converted 0_template copy 2.ipynb.
Converted 0_template copy 3.ipynb.
Converted 0_template copy.ipynb.
Converted 0_template.ipynb.
Converted aiogram_bot.ipynb.
Converted classifying_glovo_images.ipynb.
Converted foods_prompted_images_cleaning.ipynb.
Converted inference.ipynb.
Converted inference_food_categories.ipynb.
Converted inline_kb.ipynb.
Converted new_match_dataset.ipynb.
Converted recepie_scraper.ipynb.
Converted stats.ipynb.
Converted testset_exploration.ipynb.
Converted usda_parsing.ipynb.
